## Introdução

No âmbito da cadeira de Tópicos Avançado em Inteligência Artificial foi nos propostos fazer um trabalho em que o objetivo era comparar a tecnologia do deep network num dataset, para ver qual técnicas seriam úteis e para saber como funciona a sua pesquisa

In [ ]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import numpy as np

from sklearn.svm import SVC
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier


from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping

## Importação dos dados

Utilizamos a biblioteca do pandas para fazer a leitura dos dados que se encontram dentro do ficheiro data.csv.
Contendo neste ficheiro os dados registados nos pacientes com cancro.<br>
Foi feito um ligeiro tratamento de dados, retirando da tabela na coluna id algumas linhas no ficheiro que continha valores Unnamed, e também foi transformando os valores da coluna diagnosis, que continham os valores M e B que correspondiam respetivamente maligno e benigno, para o formato 0 e 1, passando os valores malignos para 0 e benignos para 1. 

In [3]:
df = pd.read_csv('data.csv')
df = df.drop(['id', 'Unnamed: 32'], axis=1)
df.diagnosis = df.diagnosis.transform(lambda x: 0 if x == 'M' else 1)

## Normalização dos dados

In [60]:
col = df.columns
x = df.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df = pd.DataFrame(x_scaled, columns=col)

## Separação dos dados em treino e teste

Foi feito uma separação dos dados sendo os dados separados em X_train e y_train, que são os dados que serão utilizados para treinar os vários modelos que irão ser utilizados, e X_test e y_test, que serão os dados que serão utilizados para fazer o teste dos vários modelos para poder saber a performance de cada um.<br>
Sendo colocados nas variaveis X_train e X_test os valores do dataset sem a coluna diagnosis, e nas variáveis y_train e y_test somente os valores da coluna diagnosis que será a coluna que queremos descobrir o resultado.

In [36]:
y = df.diagnosis.values
X = df.drop(['diagnosis'], axis=1).values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

## Create SVM

Foi utilizado uma svm com kernel linear da biblioteca sklearn.svm.<br>
Os resultados foram bastante positivos tendo uma accuracy superior a 95%. 

In [22]:
clf = SVC(kernel='linear')
clf.fit(X_train, y_train)
pred = clf.predict(X_test)

pred = clf.predict(X_test)
c = 0
for i in range(len(pred)):
    if(y_test[i] == pred[i]):
        c+=1
        
print((c/len(pred)*100))

98.24561403508771


## Random Forest

Foi utilizado uma randomforest tendo a profundidade maxima de 2 e o random_state 0 e foi importado da biblioteca sklearn.ensemble import RandomForestClassifier.
<br>
Os resultados foram também positivos, tendo uma accuracy superior a 91%.

In [29]:
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X_train, y_train)

pred = clf.predict(X_test)
c = 0
for i in range(len(pred)):
    if(y_test[i] == pred[i]):
        c+=1

print((c/len(pred)*100))

94.15204678362574


## Decision Tree

Foi utilizado uma decision tree e foi importado da biblioteca sklearn import tree.
<br>
Os resultados foram bastante bons embora tenha sido o modelo com menor performance do que os valores anteriores tendo uma accuracy superior a 89%.

In [37]:
clf = tree.DecisionTreeClassifier()
clf.fit(X_train, y_train)

pred = clf.predict(X_test)
c = 0
for i in range(len(pred)):
    if(y_test[i] == pred[i]):
        c+=1

print((c/len(pred)*100))

92.98245614035088


## KERAS

Foi utilizado o modelo sequential para a criação do deep network, sendo esse modelo um conjunto linear de camadas.
<br>
Adicionamos 6 camadas ao modelo, como a função relu serve de uma boa base de criação de uma deep network começamos no inicio com ela, no fim fizemos alguns testes com a função de sigmoid,tanh e LeakyRelu mas como a performance não melhorou e em alguns casos demora mais tempo para construir o modelo, resolvemos continuar com o relu como função de ativação do nosso modelo, em cinco camadas das 6. Utilizamos valores de neurónios alternados tendo no máximo 50, porque o nosso dataset não continha uma grande quantidade de dados por isso achamos não ser necessários possuir tantos neurónios.<br>
No último neurónio utilizamos a função de probabilidade softmax porque foi a nossa função escolhida inicialmente e a performance não melhorou em relação as outras funções de probabilidades.

In [55]:
from keras.layers import LeakyReLU
model = Sequential()
# train_y = to_categorical(df.High)

n_cols = len(df.columns) -1

model.add(Dense(30, activation='relu', input_shape=(n_cols,)))
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='softmax'))

Compilamos o modelo utilizando o adam como o optimizer, tendo como função de loss o mean_squared_error e utilizamos como métrica a accuracy.

In [56]:
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

Colocamos um monitor earlystopping para que o modelo parasse quando via que não poderia melhorar, evitando assim overfit.<br>
Treinamos o modelo com as dataset de treino X_train e y_train definidos nos passos anteriores.

In [61]:
early_stopping_monitor = EarlyStopping(patience=3)

m=model.fit(X_train, y_train, validation_split=0.2, epochs=30, callbacks=[early_stopping_monitor])
#m=model.fit(X_train, y_train, validation_split=0.2, epochs=30)

Train on 318 samples, validate on 80 samples
Epoch 1/30
318/318 [==============================] - 0s 79us/step - loss: 0.3553 - accuracy: 0.6447 - val_loss: 0.3500 - val_accuracy: 0.6500
Epoch 2/30
318/318 [==============================] - 0s 95us/step - loss: 0.3553 - accuracy: 0.6447 - val_loss: 0.3500 - val_accuracy: 0.6500
Epoch 3/30
318/318 [==============================] - 0s 74us/step - loss: 0.3553 - accuracy: 0.6447 - val_loss: 0.3500 - val_accuracy: 0.6500
Epoch 4/30
318/318 [==============================] - 0s 87us/step - loss: 0.3553 - accuracy: 0.6447 - val_loss: 0.3500 - val_accuracy: 0.6500


## Conclusão

Como constatado pelos testes feitos utilizando os diferentes modelos de machine learning, o modelo que teve a melhor performance foi o svm e o pior modelo foi o do deep learning tendo aproximadamente uma accuracy de apenas 64%.<br>
Acreditamos que o modelo de deep learning não teve grande êxito por termos usado um dataset com muito poucos dados, sendo o deep learning mas efetivo em datasets com uma grande quantidade de dados.